In [1]:
pip install langgraph langchain-google-genai langchain-community faiss-cpu pandas

In [2]:
import os
import pandas as pd

In [4]:
# --- 1. SETUP API KEY ---
# Replace "PASTE_YOUR_KEY_HERE" with your actual AIza... key
os.environ["GOOGLE_API_KEY"] = "Your_API_KEY"

In [5]:
import os
import getpass
import pandas as pd
from typing import Annotated, TypedDict, List
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END, add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command

In [6]:
# --- 1. SETUP (Same as before) ---
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("API Key: ")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# ERP Data
data = {
    "product_id": ["GPU-X100", "GPU-X100", "CPU-Z50"],
    "location": ["Taiwan", "Germany", "California"],
    "stock": [0, 50, 100]
}
df_inventory = pd.DataFrame(data)

# News Data (Vector Store)
news_feed = [
    "URGENT: Super Typhoon 'Kuna' is hitting Taiwan. Ports closed.",
    "Strike Update: Germany logistics are operating normally."
]
vector_store = FAISS.from_texts(news_feed, embedding=embeddings)
retriever = vector_store.as_retriever()

In [7]:
# --- 2. DEFINE THE TOOL (For the Agent) ---

@tool
def check_external_risks(location: str):
    """
    Checks for logistical risks (weather, strikes, wars) in a specific location.
    Use this tool whenever a shipping location is identified.
    """
    print(f"🌍 [Tool] Checking news for: {location}")
    docs = retriever.invoke(location)
    risk_info = "\n".join([doc.page_content for doc in docs])

    # --- HITL: The Safety Check inside the Tool ---
    if "Typhoon" in risk_info or "Closed" in risk_info:
        print(f"\n⚠️ CRITICAL RISK DETECTED: {risk_info}")

        # Pause and ask human
        decision = interrupt(f"Risk found in {location}. Authorize use of this info? (yes/no)")

        if decision.lower() == "no":
            return "Risk check aborted by Human Security Officer."

    return f"Risk Report for {location}: {risk_info}"

In [8]:
# --- 3. DEFINE STATE (Matching Blueprint) ---
class AgentState(TypedDict):
    messages: Annotated[list, add_messages] # Chat History
    erp_context: str                        # Data found in DB

In [9]:
# --- 4. DEFINE NODES ---

def lookup_inventory(state: AgentState):
    """Deterministic Node: Queries Pandas first."""
    last_message = state["messages"][-1]
    query = last_message.content

    print(f"📦 [Database] Scanning for products in: '{query}'")

    # Simple extraction logic (heuristic for demo)
    product_id = "Unknown"
    if "GPU" in query: product_id = "GPU-X100"
    elif "CPU" in query: product_id = "CPU-Z50"

    # Database Lookup
    row = df_inventory[df_inventory["product_id"] == product_id]

    if row.empty:
        context = "System Notification: Product not found in database."
    else:
        # We take the first match
        match = row.iloc[0]
        context = f"ERP Data: Found {match['product_id']} in {match['location']} with Stock={match['stock']}."

    print(f"   -> {context}")

    # We allow the state to update, but we DON'T return a message to the user yet.
    # We pass this 'context' silently to the Agent node.
    return {"erp_context": context}

def agent_node(state: AgentState):
    """The Brain: Decides whether to answer or call the Risk Tool."""

    # Inject the ERP context into the prompt systematically
    system_prompt = SystemMessage(content=f"""
    You are a Supply Chain Sentinel.

    CURRENT ERP STATUS:
    {state.get('erp_context', 'No data checked yet')}

    INSTRUCTIONS:
    1. If the ERP status shows a location, you MUST check for risks using the 'check_external_risks' tool.
    2. If stock is 0, warn the user but STILL check risks (to see if restock is possible).
    3. Synthesize the final answer based on stock + risk.
    """)

    # Bind tools to the LLM (This gives it the 'power' to call functions)
    model_with_tools = llm.bind_tools([check_external_risks])

    # Combine System Prompt + Conversation History
    messages = [system_prompt] + state["messages"]

    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

In [10]:
# --- 5. BUILD GRAPH (The Blueprint Flow) ---

workflow = StateGraph(AgentState)

# Nodes
workflow.add_node("database", lookup_inventory)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", ToolNode([check_external_risks])) # Standard ToolNode

# Edges
workflow.set_entry_point("database")
workflow.add_edge("database", "agent")

# Conditional Edge: The "Reasoning Loop"
# If Agent calls a tool -> Go to 'tools'
# If Agent just talks -> Go to END
workflow.add_conditional_edges(
    "agent",
    tools_condition,
)

workflow.add_edge("tools", "agent") # Loop back to agent after tool usage

# Compile
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
# --- 6. EXECUTION ---

config = {"configurable": {"thread_id": "blueprint_demo_1"}}

print("\n🚀 Blueprint Agent Started...")

# User asks about the risky item
inputs = {
    "messages": [HumanMessage(content="We need to ship GPU-X100 immediately.")],
    "erp_context": ""
}


🚀 Blueprint Agent Started...


In [12]:
# Run
result = app.invoke(inputs, config=config)

📦 [Database] Scanning for products in: 'We need to ship GPU-X100 immediately.'
   -> ERP Data: Found GPU-X100 in Taiwan with Stock=0.
🌍 [Tool] Checking news for: Taiwan

⚠️ CRITICAL RISK DETECTED: URGENT: Super Typhoon 'Kuna' is hitting Taiwan. Ports closed.
Strike Update: Germany logistics are operating normally.


In [13]:
# Note: The code will PAUSE inside the tool if 'Typhoon' is found.
# You will see the interrupt prompt.

print("\n✅ Final Response:")
print(result["messages"][-1].content)


✅ Final Response:
[{'type': 'text', 'text': 'GPU-X100 is currently out of stock in Taiwan. I will check for external risks in Taiwan to see if restock is possible.', 'extras': {'signature': 'CocGAXLI2nw7OtxqO4c4YlKeRcAFfm0I5ynsEPaliGU1WF719qz7uBB1bumiobC2a1kjV7z3z61UFbPi83+QRAalzq5bSth0h4YckK7vZC3gcpDKp2abk6is/DbUmDxIl1XCgiVGN3zqgYso/AHVfifg3ubBDURBD3n6EchTKml4yPHZS+kL+R0JW91RotN717D3ldK8TUM63z3Fjxb7J5DKt/HO751wlla9olDFx/c8I0fwWIeTnfhu2JhBhrZwG6GypSVNcMV4SfrWvbKgA3pGtJ9nxZhOWMSBhfa/nzgIP93BUjreG6Qcxm6B3Vj2PqHaawRnHP0lN0zYzlx1iTp4rQo6FGcS4CdN5ntOcTdLfJZ4zRmWDu+Ye4XoW1G0Nc47JnjtK4OrxCjwXEcewvhY53swM2ugW5c3piDBfHXURZDj0O4Q+WtaAYkb99I2GY+a806Rrwv9omYqA5hSogK1W0T8CfbVOVweF/0QiY+Y6hMHOb7E2I0NHURQbQWN/DajNIXp1/bOueKa30SxJhYKQTUCn6g4MiQ9VgiEizTjPp60vdFNFiRKnTW5hIg2j8Wbxi429sBoPapSbhJrFVx+FolILZWcSEYVnkleZUcIKPqs6oSGTzoU1kRWyg0YUWtTN+S7pyUKy4kEumkwky8kMjUSCo3d+tzBWObE3HuN5CFZSh/EJkgGbhPVmo0fv8+InpFUidXQk8fJGeZGE9yNSeSqZhmUrX8cBwuxLerbIiI9M645AECcuLEuo9RVe8ndSqT+eRZk2py+KuujDm/erezP8hO5IcqexQJY

In [14]:
# Run
result = app.invoke(inputs, config=config)

# Note: The code will PAUSE inside the tool if 'Typhoon' is found.
# You will see the interrupt prompt.

print("\n✅ Final Response:")
print(result["messages"][-1].content)

📦 [Database] Scanning for products in: '[{'type': 'text', 'text': 'GPU-X100 is currently out of stock in Taiwan. I will check for external risks in Taiwan to see if restock is possible.', 'extras': {'signature': 'CocGAXLI2nw7OtxqO4c4YlKeRcAFfm0I5ynsEPaliGU1WF719qz7uBB1bumiobC2a1kjV7z3z61UFbPi83+QRAalzq5bSth0h4YckK7vZC3gcpDKp2abk6is/DbUmDxIl1XCgiVGN3zqgYso/AHVfifg3ubBDURBD3n6EchTKml4yPHZS+kL+R0JW91RotN717D3ldK8TUM63z3Fjxb7J5DKt/HO751wlla9olDFx/c8I0fwWIeTnfhu2JhBhrZwG6GypSVNcMV4SfrWvbKgA3pGtJ9nxZhOWMSBhfa/nzgIP93BUjreG6Qcxm6B3Vj2PqHaawRnHP0lN0zYzlx1iTp4rQo6FGcS4CdN5ntOcTdLfJZ4zRmWDu+Ye4XoW1G0Nc47JnjtK4OrxCjwXEcewvhY53swM2ugW5c3piDBfHXURZDj0O4Q+WtaAYkb99I2GY+a806Rrwv9omYqA5hSogK1W0T8CfbVOVweF/0QiY+Y6hMHOb7E2I0NHURQbQWN/DajNIXp1/bOueKa30SxJhYKQTUCn6g4MiQ9VgiEizTjPp60vdFNFiRKnTW5hIg2j8Wbxi429sBoPapSbhJrFVx+FolILZWcSEYVnkleZUcIKPqs6oSGTzoU1kRWyg0YUWtTN+S7pyUKy4kEumkwky8kMjUSCo3d+tzBWObE3HuN5CFZSh/EJkgGbhPVmo0fv8+InpFUidXQk8fJGeZGE9yNSeSqZhmUrX8cBwuxLerbIiI9M645AECcuLEuo9RVe8ndSqT+eRZk2py+Kuu